# 打新资金追踪

## 1. 跟踪当日上dex的IDO  

通过transpose平台
```sql
select * from ethereum.dex_pools WHERE 
        created_timestamp > (now() - INTERVAL '1 day')
```



## 2. 打新账号追踪 


追踪每个池子的前5笔买入

```sql
WITH
    ExcludedAddresses AS (
        SELECT
            UNNEST(
                ARRAY[
                    '0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2',
                    '0xdAC17F958D2ee523a2206206994597C13D831ec7',
                    '0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48'
                ]
            ) AS address
    ),
    np AS (
        SELECT
            *
        FROM
            ethereum.dex_pools
        WHERE
            created_timestamp > (NOW() - INTERVAL '1 day')
    ),
    NumberedTransactions AS (
        SELECT
            *,
            ROW_NUMBER() OVER (
                PARTITION BY contract_address
                ORDER BY timestamp ASC -- Assumes a timestamp column to order the transactions
            ) AS rn
        FROM
            ethereum.dex_swaps
        WHERE
            contract_address IN (
                SELECT
                    contract_address
                FROM
                    np
            )
            AND to_token_address NOT IN (SELECT address FROM ExcludedAddresses)
    )
SELECT
    *
FROM
    NumberedTransactions
WHERE
    rn BETWEEN 1 AND 10;
```